In [8]:
import torch
import torchaudio
import json
from jiwer import wer
from transformers.file_utils import cached_path, hf_bucket_url
from importlib.machinery import SourceFileLoader
from transformers import Wav2Vec2ProcessorWithLM
from IPython.lib.display import Audio
import torchaudio
import torch
from pyctcdecode import BeamSearchDecoderCTC

In [9]:
data_1, sample_rate = torchaudio.load("data1.wav")
data_2, sample_rate = torchaudio.load("data2.wav")
output, sample_rate = torchaudio.load("output.wav")

path = "/project/data_asr/CHiME5/data/CHiME5/audio/dev/"
micp5, sample_rate = torchaudio.load(path+"S02_P05.wav")
micp5 = torch.narrow(micp5, 1, 0, 284929*10)
micp6, sample_rate = torchaudio.load(path+"S02_P06.wav")
micp6 = torch.narrow(micp6, 1, 0, 284929*10)

## ASR

In [10]:
# Get Model
model_name = "nguyenvulebinh/iwslt-asr-wav2vec-large-4500h"
model = SourceFileLoader("model", cached_path(hf_bucket_url(model_name,filename="model_handling.py"))).load_module().Wav2Vec2ForCTC.from_pretrained(model_name)
processor = Wav2Vec2ProcessorWithLM.from_pretrained(model_name)

Only 0 unigrams passed as vocabulary. Is this small or artificial data?


In [11]:
input_1 = processor.feature_extractor(data_1[0], sampling_rate=16000, return_tensors='pt')
output_1 = model(**input_1)
output_1 = processor.decode(output_1.logits.cpu().detach().numpy()[0], beam_width=100).text

In [12]:
input_2 = processor.feature_extractor(data_2[0], sampling_rate=16000, return_tensors='pt')
output_2 = model(**input_2)
output_2 = processor.decode(output_2.logits.cpu().detach().numpy()[0], beam_width=100).text


In [13]:
result = processor.feature_extractor(output[0], sampling_rate=16000, return_tensors='pt')
out = model(**result)
out = processor.decode(out.logits.cpu().detach().numpy()[0], beam_width=100).text

In [14]:
micp5 = processor.feature_extractor(micp5[0], sampling_rate=16000, return_tensors='pt')
micp5 = model(**micp5)
micp5 = processor.decode(micp5.logits.cpu().detach().numpy()[0], beam_width=100).text

In [15]:
micp6 = processor.feature_extractor(micp6[0], sampling_rate=16000, return_tensors='pt')
micp6 = model(**micp6)
micp6 = processor.decode(micp6.logits.cpu().detach().numpy()[0], beam_width=100).text

In [1]:
# Write Output into Txt File

## Evaluation

In [16]:
# Getting ground truth for S02\n",
with open('./S02.json', 'r') as f:
    text = json.load(f)
    
transcript = ""
for part in text:
    transcript += str(part["words"]) + " "

transcript = transcript[:300]
transcript

"[laughs] It's the blue, I think. I think. Okay. [laughs] Let's do lunch. [laughs] Let's do lunch! [laughs] Okay, so here's the pie. Can I help with anything? Wow, that looks good. Um yeah. [noise] Since we Hales, did you put the is the oven heating, or did you put it off cuz it was too hot? Okay. No"

In [18]:
print("P5 Mic WER:", wer(transcript,micp5))
print("P6 Mic WER:", wer(transcript,micp6))

print("Array 1 WER:", wer(transcript,output_1))
print("Array 2 WER:", wer(transcript,output_2))
print("Beamformer WER:", wer(transcript,out))
# ground truth, hypothesis"

#Better performance if short segments (~10s-15s)

#1 audio

#dataset ICSI Meeting
#/dev/shm

P5 Mic WER: 0.9661016949152542
P6 Mic WER: 0.847457627118644
Array 1 WER: 1.0
Array 2 WER: 1.0
Beamformer WER: 0.9830508474576272


In [24]:
print(out)

the
